In [69]:
# First Always set logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Covnert tweets to vectors #

### Read and create CSV corpus class ###

In [70]:
import csv


class TrainCorpus(object):
    def __init__(self, file_path):
        self.file_path = file_path
        self.gender_index = 5
        self.description_index = 10
        self.tweet_index = 19

    def __iter__(self):
        with open(self.file_path, newline='') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',', doublequote=False, quoting=csv.QUOTE_NONE)
            for row in csv_reader:
                if len(row) < 20:
                    continue
                yield [row[self.gender_index], row[self.description_index], row[self.tweet_index]]

### Preprocess data class ###

In [71]:
from gensim.utils import tokenize
from nltk.corpus import stopwords
import re


class PreprocessedCorpus(object):
    def __init__(self, corpus):
        self.corpus = corpus
        
    def __iter__(self):
        for line in train_corpus:
            gender = line[0]
            description = line[1]
            text = line[2]
            if gender not in ['male', 'female', 'brand']:
                continue

            # remove urls
            cleaned_descr = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', description, flags=re.MULTILINE)
            cleaned_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text, flags=re.MULTILINE)

            # tokenize
            words = list(tokenize(cleaned_descr, deacc=True))
            words.extend(tokenize(cleaned_text, deacc=True))

            # remove stop words
            filtered_words = [word for word in words if word not in stopwords.words('english')]
            
            yield [gender, filtered_words]

In [72]:
# from gensim.models import Word2Vec
# from gensim.utils import tokenize
# model = Word2Vec.load('models/...')

## Pipeline ##

In [73]:
train_corpus = TrainCorpus('train-data/gender-classifier-DFE-791531.csv')
preprocessed_corpus = PreprocessedCorpus(train_corpus)

# for line in preprocessed_corpus:
#